In [1]:
import pandas as pd

In [2]:
from google.cloud import bigquery

In [3]:
client = bigquery.Client()

query = """
SELECT ano, id_municipio, sigla_uf, abrangencia, tipo_servico, natureza_juridica, populacao_atendida_agua, populacao_atentida_esgoto, populacao_urbana, populacao_urbana_atendida_agua, populacao_urbana_atendida_esgoto, extensao_rede_agua, extensao_rede_esgoto, volume_agua_produzido, volume_agua_tratada_eta, volume_agua_consumido, volume_esgoto_coletado, volume_esgoto_tratado,  quantidade_ligacao_total_agua, quantidade_ligacao_total_esgoto
FROM basedosdados.br_mdr_snis.prestador_agua_esgoto
WHERE ano >= 2012 and ano < 2022;
"""

df_saneamento = client.query(query).to_dataframe()


In [4]:
# verifica anos disponiveis
df_saneamento['ano'].sort_values().unique()

<IntegerArray>
[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
Length: 10, dtype: Int64

In [5]:
# entendo dados
print(df_saneamento.info())
print('Descrição:\n', df_saneamento.describe())
print('Primeiras linhas:\n', df_saneamento.head())
print('últimas linhas:\n', df_saneamento.tail())
print('Shape:\n', df_saneamento.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56455 entries, 0 to 56454
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ano                               56455 non-null  Int64  
 1   id_municipio                      56455 non-null  object 
 2   sigla_uf                          56455 non-null  object 
 3   abrangencia                       56455 non-null  object 
 4   tipo_servico                      56455 non-null  object 
 5   natureza_juridica                 56455 non-null  object 
 6   populacao_atendida_agua           52133 non-null  Int64  
 7   populacao_atentida_esgoto         24976 non-null  Int64  
 8   populacao_urbana                  56455 non-null  Int64  
 9   populacao_urbana_atendida_agua    52124 non-null  Int64  
 10  populacao_urbana_atendida_esgoto  24977 non-null  Int64  
 11  extensao_rede_agua                52155 non-null  float64
 12  exte

In [6]:
# normatizando dataframe
for col in df_saneamento.select_dtypes(include='object').columns:
  df_saneamento[col] = df_saneamento[col].str.strip().str.lower()

In [8]:
#verificando nulos
df_saneamento.isnull().sum().sort_values(ascending=False)

,0
extensao_rede_esgoto,31493
populacao_atentida_esgoto,31479
populacao_urbana_atendida_esgoto,31478
quantidade_ligacao_total_esgoto,31477
volume_esgoto_coletado,31462
volume_esgoto_tratado,31457
populacao_urbana_atendida_agua,4331
populacao_atendida_agua,4322
extensao_rede_agua,4300
quantidade_ligacao_total_agua,4292


In [9]:
# tratando nulos. Existe municipios com dois ou mais prestadores de serviço, e só aprece informação do serviço prestado por cada um
df_saneamento.fillna(0, inplace=True)

In [10]:
# verificar tratamento de nulos
df_saneamento.isnull().sum().sort_values(ascending=False)

,0
ano,0
id_municipio,0
quantidade_ligacao_total_agua,0
volume_esgoto_tratado,0
volume_esgoto_coletado,0
volume_agua_consumido,0
volume_agua_tratada_eta,0
volume_agua_produzido,0
extensao_rede_esgoto,0
extensao_rede_agua,0


In [11]:
# verificação de duplicados
df_saneamento.duplicated().sum()


0

In [12]:
# padronizar coluna tipo_servico
df_saneamento['tipo_servico'] = df_saneamento['tipo_servico'].replace({'esgotos': 'esgoto'})

df_saneamento['tipo_servico'].unique()

array(['esgoto', 'agua e esgoto', 'agua'], dtype=object)

In [13]:
# buscando outro conjunto de dados
query_municipios = """
SELECT `Código Município Completo` AS id_municipio_ibge, `Nome_Município` AS `nome_municipio`
FROM `saneamentobasicobrasil-456115.saneamento_br.municipios_ibge`
"""
df_municipios = client.query(query_municipios).to_dataframe()


In [14]:
# nortatizando tipos
df_saneamento['id_municipio'] = df_saneamento['id_municipio'].astype(int)

df_saneamento = df_saneamento.merge(df_municipios, left_on='id_municipio', right_on='id_municipio_ibge', how='left')

In [15]:
pip install pandas-gbq

In [17]:
from pandas_gbq import to_gbq

# salvar
to_gbq(df_saneamento,
       destination_table='saneamento_br.dado_saneamento_limmpos', project_id='saneamentobasicobrasil-456115', if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 6043.67it/s]
